## PDF Parsing

In [93]:
import sys
sys.path.append('src')
import os 
import src.mapping as m
import src.classes
from src.classes.PDFReader import PDFReader as pdf_reader
from src.classes.Prompter import Prompter
import importlib

In [94]:
# Extract by Section
pdf_path = "20-301704.3.pdf" # ESA BL Data row 5736 - 6666

In [95]:
pdf_path2 = "20-301704.3.pdf" # ESA BL Data row 5736 - 6666

In [96]:
# Extract by Subsection
subsection_dict = pdf_reader().process_file(pdf_path)
for key, val in subsection_dict.items():
    print(key)
    print(subsection_dict[key])
    
    

1
{'1.0': ' 1.0    INTRODUCTION Partner Engineering and Science, Inc. (Partner) has performed a Phase I Environmental Site Assessment (ESA) in conformance with the scope and limitations of ASTM Practice E1527-21 and the Environmental Protection Agency Standards and Practices for All Appropriate Inquiries (AAI) (40 CFR Part 312) for the property located at 115 West Anapamu Street in Santa Barbara, Santa Barbara County, California (the “subject property”).  Any exceptions to, or deletions from, this scope of work are described in the report.', '1.1': ' 1.1    Purpose The purpose of this ESA is to identify existing or potential Recognized Environmental Conditions (as defined by ASTM Standard E1527-21) affecting the subject property that: 1) constitute or result in a material violation or a potential material violation of any applicable environmental law; 2) impose any material constraints on the operation of the subject property or require a material change in the use thereof; 3) require 

In [97]:
subsection_dict2 = pdf_reader().process_file(pdf_path2)
#for key in subsection_dict:
    #print(subsection_dict[key])

## LLM Setup + Prompt

In [98]:
prompter = Prompter()

In [99]:
%%time
datafields = m.execute()

21
{'6.0', '3.1', '6.1', '4.1', '1.5', '3.2', '2.4', '6.4', '1.0', '5.1', '3.3', '2.2', '2.3', '7.0', '6.2', '4.2', '5.2', '3.4', '6.3', '2.1', '3.5'}
CPU times: total: 93.8 ms
Wall time: 121 ms


In [100]:
for items in sorted(datafields.items()):
  # print(items)
  subsection = items[0]
  # print(subsection)
  questions = items[1]
  # print(questions)
  test = {items[0]: items[1]}
  # print(test)
  for field, question in questions.items():
    print(f"field: {field}")
    print(f"question: {question}")
    print()

field: 1po_client_scope_format
question: Client Scope Format: 

field: 1po_project_level_scope_format
question: Project Level Scope Format: 

field: 1po_scope_standard
question: Phase 1 Scope Standard:

field: 1apl_project_number
question: Project Number

field: 1bpl_project_name
question: Project Name

field: 1cpl_project_address
question: Project Address

field: 1dpl_project_city
question: Project City

field: ets_1_1ctv
question: City/Town/Village of <City>

field: 1epl_project_county
question: Project County

field: 1fpl_project_state
question: Project State

field: 1gpl_project_zip
question: Project Zip

field: 1rdd_report_due_date
question: Report Due Date

field: 1cfn_client_first
question: Client First

field: 1cln_client_last
question: Client Last

field: 1csa_client_salutation
question: Client Salutation

field: 1cco_client_company
question: Client Company

field: 1cad_client_address
question: Client Address

field: 1cci_client_city
question: Client City

field: 1cst_client_s

In [101]:
from langchain_community.llms import Ollama
llm = Ollama(model="phi3", temperature=0, num_predict=40, top_k=5, top_p=.3, mirostat_tau=0, format="json")

In [102]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

output_parser = StrOutputParser()
json_parser = JsonOutputParser()

In [103]:
import json

In [104]:
%%time
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# prompt = ChatPromptTemplate.from_template("""Retrieve the datafield defined in input within the context given.
# Anything legal can be found in the context.
# If you cannot answer the question with the context, respond with N/A.

# <context>
# {context}
# </context>

# {input} is the prompt.
# Again, if you cannot answer the question respond N/A.
# Simply respond with only the answer that matches the datafield.
# There must only be one JSON object.
# Do not include the datafield in your response. Keep the key for the json exactly the same as the input.
# """)

#document_chain = create_stuff_documents_chain(llm, prompt, output_parser=output_parser)

CPU times: total: 0 ns
Wall time: 0 ns


In [105]:
from langchain.chains import create_retrieval_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser


# System Chat History
#context_system_prompt = """Given a chat history and the latest question \
#which might reference context in the chat history, answer the question. """
context_system_prompt = """Retrieve the datafield defined in input within the context given. 
Anything legal can be found in the context and chat history.
If you cannot answer the question with the context, respond with N/A.

<context>
{context}
</context>

{input} is the prompt.
Again, if you cannot answer the question respond N/A.
Simply respond with only the answer that matches the datafield.
There must only be one JSON object.
Do not include the datafield in your response. Keep the key for the json exactly the same as the input.
"""
# ChatPromptTemplate.from_template()

context_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", context_system_prompt),
        MessagesPlaceholder("chat_history", optional=True)#,
        ,("human", "{input}")
    ]
)


qa_system_prompt = context_system_prompt #"""Reading through a PDF and giving only the answers found within it. If you do not know an answer, you respond N/A."""


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history")#,
        ,("human", "{input}")
    ]
)


qa_document_chain = qa_prompt | llm | JsonOutputParser()
history_aware_retriever = context_prompt | llm | JsonOutputParser()
retrieval_chain = create_retrieval_chain(
    history_aware_retriever, qa_document_chain)

chat_history = []


In [106]:
#response = retrieval_chain.invoke({'input':input, 'chat_history':chat_history})


In [107]:
%%time
from langchain_core.documents import Document
answers_dict = {}

for subsection, field_to_questions in sorted(datafields.items()):
    subsection_answers = {}
    
    # TAKE THIS OUT: JUST FOR DEMO PURPOSES:
    if subsection:      # (subsection == *selected section*)
        print("--------------------------------------------------------")
        print(f"Current Subsection: {subsection}")
        section_number = subsection.split('.')[0]
        print(f"Current Section: {section_number}")
        for field, question in field_to_questions.items():
            # print(item[subsection][field])
            # for key in subsection_dict.keys():
            #     print('.' + key + '.')
            #     print(type(key))
                
            #print(subsection_dict.keys())
            # section = subsection.
            #print(' 3' in subsection_dict.keys())
            #print(item[subsection][field])
            try:
                # print(subsection_dict[section_number].keys())
                subsection_context = subsection_dict[section_number][subsection]
                print("SUBSECTION CONTEXT: ", subsection_context)
                print(f"> field: {field}")
                print(f"> question: {question}")
                # -----------------------------INVOKE LLM---------------------------------------
                output_dict = retrieval_chain.invoke({'input':question, 'context': [Document(page_content=subsection_context)], 
                                                    'chat_history':chat_history})
                
                # -----------------------------INVOKE LLM---------------------------------------
                # print(f"Model Output: {output_dict}")
                key = list(output_dict['answer'].keys())[0]
                #real_output_dict = {}
                subsection_answers[question] = output_dict['answer'][key]
                answer = subsection_answers[question]
                print(f'> Answer: {answer}\n')
                #print(output_dict.keys[0])
            except Exception as e:
                print("!!!!!!!!!!!!!!!")
                print(f'Error in Subsection {subsection}: {e}')
                print("!!!!!!!!!!!!!!!")
            #answers_dict[field] = output_dict[item[subsection][field]]
            #print(answers_dict[field])
    answers_dict[subsection] = subsection_answers
print("answers dict is ", answers_dict)
# Write to JSON
with open("data/processed/phi3_output2.json", "w") as outfile: 
    json.dump(answers_dict, outfile)

--------------------------------------------------------
Current Subsection: 1.0
Current Section: 1
SUBSECTION CONTEXT:   1.0    INTRODUCTION Partner Engineering and Science, Inc. (Partner) has performed a Phase I Environmental Site Assessment (ESA) in conformance with the scope and limitations of ASTM Practice E1527-21 and the Environmental Protection Agency Standards and Practices for All Appropriate Inquiries (AAI) (40 CFR Part 312) for the property located at 115 West Anapamu Street in Santa Barbara, Santa Barbara County, California (the “subject property”).  Any exceptions to, or deletions from, this scope of work are described in the report.
> field: 1po_client_scope_format
> question: Client Scope Format: 
> Answer: Partner Engineering and Science, Inc. has performed a Phase I Environmental Site Assessment (ESA) in conformance with A

SUBSECTION CONTEXT:   1.0    INTRODUCTION Partner Engineering and Science, Inc. (Partner) has performed a Phase I Environmental Site Assessment (E

In [62]:
# from langchain_community.llms import Ollama
# system = """Reading through a PDF and giving only the answers found within it. If you do not know an answer, you respond N/A."""
# template = """[INST] {{ .System }} {{ .Prompt}} [/INST]"""
# stop_tokens = ["[INST]", "[/INST]"]
# llm = Ollama(model="phi3", system=system, template=template,
#              stop=stop_tokens, temperature=0, num_predict=20, top_k=3, top_p=.03, mirostat_tau=1, format="json")
# document_chain = create_stuff_documents_chain(llm, prompt, output_parser=json_parser)

## Responses

In [63]:
# fields_dict = {'ets_2p_32a': 'Name of Assessor/Appraisal District Agency', 
#                'ets_2p_33a': 'Property Legal Description', 
#                'ets_2p_34a': 'Property Owner Name'}
# #               'ets_2p_35a': "When did the owner acquire the property?",
#  #              "ets_2p_36a": "What was the source of the property owner name and acquisition date?"}

In [65]:
# %%time
# from langchain_core.documents import Document
# answers_dict = {}

# for items in datafields.items():
#     try:
    
#         item = {items[0]: items[1]} 
#         # Item is of shape : {'1.1' : {'ets_1p_32a': 'Purpose:', 'esa_foia_1': 'ESA Jurisdiction 1', 
#         #                     'esa_foia_contact_1': 'ESA Jurisdiction Contact 1', 'ESA Jurisdiction Contact 3'}}
#         # print("ITEM IS " , item)
#     except Exception as e:
#         print('ERROR')
#         print(items)
#         print(e)
#         print()
#     for subsection in item:
#         print("SUBSECTION:" ,subsection)
#         print(subsection == '.1')
#         section_number = subsection.split('.')[0]
#         print(section_number)
#         # TAKE THIS OUT: JUST FOR DEMO PURPOSES:
#         if not section_number == '6':
#             for field in item[subsection]:
#                 print(field)
#                 # print(item[subsection][field])
#                 # for key in subsection_dict.keys():
#                 #     print('.' + key + '.')
#                 #     print(type(key))
                    
#                 #print(subsection_dict.keys())
#                # section = subsection.
#                 #print(' 3' in subsection_dict.keys())
#                 #print(item[subsection][field])
#                 try:
#                     print(subsection_dict[section_number].keys())
#                     subsection_context = subsection_dict[section_number][subsection]
#                     print("SUBSECTION CONTEXT: ", subsection_context)
#                     print("INPUT QUESTION IS ", item[subsection][field])
#                     output_dict = document_chain.invoke({
#                         "input": item[subsection][field],
#                         "context": [Document(page_content=subsection_context)]
#                     })
#                     key = list(output_dict.keys())[0]
#                     #real_output_dict = {}
#                     answers_dict[field] = output_dict[key]
#                     print('MODEL OUTPUTS' , output_dict)
#                     print('PROCESSED OUTPUT' , field, ":", answers_dict[field])
#                     #print(output_dict.keys[0])
#                 except Exception as e:
#                     print('exception: ', e)
#                 #answers_dict[field] = output_dict[item[subsection][field]]
#                 #print(answers_dict[field])
# print("answers dict is ", answers_dict)
# with open("data/processed/phi3_output.json", "w") as outfile: 
#     json.dump(answers_dict, outfile)

SUBSECTION: 5.3.1
False
5
ets_5_96a
dict_keys(['5.0', '5.1', '5.1.1', '5.1.2', '5.1.3', '5.1.4', '5.1.5', '5.2', '5.2.1', '5.2.2', '5.2.3', '5.2.4', '5.2.5', '5.2.6'])
exception:  '5.3.1'
ets_5_97a
dict_keys(['5.0', '5.1', '5.1.1', '5.1.2', '5.1.3', '5.1.4', '5.1.5', '5.2', '5.2.1', '5.2.2', '5.2.3', '5.2.4', '5.2.5', '5.2.6'])
exception:  '5.3.1'
ets_5_98a
dict_keys(['5.0', '5.1', '5.1.1', '5.1.2', '5.1.3', '5.1.4', '5.1.5', '5.2', '5.2.1', '5.2.2', '5.2.3', '5.2.4', '5.2.5', '5.2.6'])
exception:  '5.3.1'
ets_5_99a
dict_keys(['5.0', '5.1', '5.1.1', '5.1.2', '5.1.3', '5.1.4', '5.1.5', '5.2', '5.2.1', '5.2.2', '5.2.3', '5.2.4', '5.2.5', '5.2.6'])
exception:  '5.3.1'
ets_5_100a
dict_keys(['5.0', '5.1', '5.1.1', '5.1.2', '5.1.3', '5.1.4', '5.1.5', '5.2', '5.2.1', '5.2.2', '5.2.3', '5.2.4', '5.2.5', '5.2.6'])
exception:  '5.3.1'
ets_5_101a
dict_keys(['5.0', '5.1', '5.1.1', '5.1.2', '5.1.3', '5.1.4', '5.1.5', '5.2', '5.2.1', '5.2.2', '5.2.3', '5.2.4', '5.2.5', '5.2.6'])
exception:  '5.3.1'


KeyboardInterrupt: 

In [110]:
%%time
from langchain_core.documents import Document

print(type(document_chain.invoke({
    "input": "Property Legal Description",
    "context": [Document(page_content=one_subsection)]
})))


KeyboardInterrupt



In [23]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Property Owner Name",
    "context": [Document(page_content=one_subsection)]
})


KeyboardInterrupt



In [24]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "When did the owner acquire the property?",
    "context": [Document(page_content=one_subsection)]
})

NameError: name 'one_subsection' is not defined

In [ ]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "What was the source of the property owner name and acquisition date?",
    "context": [Document(page_content=one_subsection)]
})